# Reconocedor de billetes

In [12]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

In [17]:
denominaciones = ["Veinte", "Quinientos"]

masks = []
masks.append([[0, 0, 39], [106, 92, 130]]) #20
masks.append([[0, 0, 34], [91, 71, 143]]) #500

In [14]:
def funcion(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    ret, imgt = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    cv2.imshow("Image threshold", imgt)
    countours, hierarchy = cv2.findContours(imgt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rectangles = [cv2.boundingRect(countour) for countour in countours]
    sizeofImage = (400, 300)
    for rect in rectangles:
        if rect[2] > 100 and rect[3] > 50 and rect[2] < 620:
            imgn = img[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]]
            imgn = cv2.resize(imgn, (sizeofImage[0], sizeofImage[1]))
            
            imgn = cv2.cvtColor(imgn, cv2.COLOR_BGR2HSV)
            iWinner, p = 0, 0
            for i, mask in enumerate(masks):
                msk = cv2.inRange(imgn, np.array(mask[0]), np.array(mask[1]))
                iMsk = cv2.bitwise_and(imgn, imgn, mask=msk)
                iMsk = cv2.cvtColor(iMsk, cv2.COLOR_HSV2BGR)
                pOn = np.count_nonzero(iMsk != 0)
                if pOn > p:
                    p = pOn
                    iWinner = i
                    print("New Winner:", i, pOn)
            imgn = cv2.cvtColor(imgn, cv2.COLOR_HSV2BGR)            
            cv2.imshow("Image rect", imgn)

            cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (255, 0, 0), 2)
            cv2.putText(img, denominaciones[iWinner], (rect[0], rect[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 0), 3, cv2.LINE_AA)
    return img

In [21]:
masks = []

# A required callback method that goes into the trackbar function.
def nothing(x):
    pass

# Initializing the webcam feed.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Now create 6 trackbars that will control the lower and upper range of 
# H,S and V channels. The Arguments are like this: Name of trackbar, 
# window name, range,callback function. For Hue the range is 0-179 and
# for S,V its 0-255.

i, thearray = 0, []
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)

while True: 
    if len(masks) == len(denominaciones): break
    cv2.setTrackbarPos("L - H", "Trackbars", 0)
    cv2.setTrackbarPos("L - S", "Trackbars", 0)
    cv2.setTrackbarPos("L - V", "Trackbars", 0)
    cv2.setTrackbarPos("U - H", "Trackbars", 179)
    cv2.setTrackbarPos("U - S", "Trackbars", 255)
    cv2.setTrackbarPos("U - V", "Trackbars", 255)

    while True:

        # Start reading the webcam feed frame by frame.
        ret, frame = cap.read()
        if not ret: break
        # Flip the frame horizontally (Not required)
        frame = cv2.flip( frame, 1 ) 

        # Convert the BGR image to HSV image.
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Get the new values of the trackbar in real time as the user changes 
        # them
        l_h = cv2.getTrackbarPos("L - H", "Trackbars")
        l_s = cv2.getTrackbarPos("L - S", "Trackbars")
        l_v = cv2.getTrackbarPos("L - V", "Trackbars")
        u_h = cv2.getTrackbarPos("U - H", "Trackbars")
        u_s = cv2.getTrackbarPos("U - S", "Trackbars")
        u_v = cv2.getTrackbarPos("U - V", "Trackbars")

        # Set the lower and upper HSV range according to the value selected
        # by the trackbar
        lower_range = np.array([l_h, l_s, l_v])
        upper_range = np.array([u_h, u_s, u_v])

        # Filter the image and get the binary mask, where white represents 
        # your target color
        mask = cv2.inRange(hsv, lower_range, upper_range)

        # You can also visualize the real part of the target color (Optional)
        res = cv2.bitwise_and(frame, frame, mask=mask)

        # Converting the binary mask to 3 channel image, this is just so 
        # we can stack it with the others
        mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        # stack the mask, orginal frame and the filtered result
        stacked = np.hstack((mask_3,frame,res))

        # Show this stacked frame at 40% of the size.
        cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))

        # If the user presses ESC then exit the program
        key = cv2.waitKey(1)
        if key == 27:
            break

        # If the user presses `s` then print this array.
        if key == ord('s'):

            thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
            print(thearray)

            # Also save this array as penval.npy
            #np.save('hsv_value',thearray)
            
            break
        
    masks.append(thearray)

    
# Release the camera & destroy the windows.    
cap.release()
cv2.destroyAllWindows()

In [20]:
cam = cv2.VideoCapture(0)
while True:
    val, img = cam.read()
    img = funcion(img)
    cv2.imshow("Image",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

New Winner: 0 34542
New Winner: 1 59268
New Winner: 0 154296
New Winner: 0 94254
New Winner: 1 101547
New Winner: 0 58554
New Winner: 1 83826
New Winner: 0 167655
New Winner: 0 90189
New Winner: 1 97416
New Winner: 0 58554
New Winner: 1 83826
New Winner: 0 167655
New Winner: 0 90189
New Winner: 1 97416
New Winner: 0 58554
New Winner: 1 83826
New Winner: 0 167655
New Winner: 0 90189
New Winner: 1 97416
New Winner: 0 177933
New Winner: 0 76278
New Winner: 1 82794
New Winner: 0 177933
New Winner: 0 76278
New Winner: 1 82794
New Winner: 0 177933
New Winner: 0 76278
New Winner: 1 82794
New Winner: 0 180489
New Winner: 0 71193
New Winner: 1 77325
New Winner: 0 180489
New Winner: 0 71193
New Winner: 1 77325
New Winner: 0 180489
New Winner: 0 71193
New Winner: 1 77325
New Winner: 0 181815
New Winner: 0 67377
New Winner: 1 71592
New Winner: 0 181815
New Winner: 0 67377
New Winner: 1 71592
New Winner: 0 181815
New Winner: 0 67377
New Winner: 1 71592
New Winner: 0 30813
New Winner: 1 59682
New Wi

New Winner: 1 185331
New Winner: 0 122484
New Winner: 1 194301
New Winner: 0 115161
New Winner: 1 188511
New Winner: 0 31731
New Winner: 1 40095
New Winner: 0 59565
New Winner: 0 23475
New Winner: 1 30090
New Winner: 0 75060
New Winner: 1 105693
New Winner: 0 31434
New Winner: 1 32442
New Winner: 0 67443
New Winner: 1 97314
New Winner: 0 39705
New Winner: 1 43467
New Winner: 0 94296
New Winner: 1 124206
New Winner: 0 33420
New Winner: 1 38118
New Winner: 0 84510
New Winner: 1 109572
New Winner: 0 25314
New Winner: 1 33000
New Winner: 0 109152
New Winner: 1 111909
New Winner: 0 21336
New Winner: 1 30141
New Winner: 0 110550
New Winner: 1 118641
New Winner: 0 15660
New Winner: 1 31569
New Winner: 0 116502
New Winner: 1 118041
New Winner: 0 13263
New Winner: 1 31662
New Winner: 0 116859
New Winner: 1 120972
New Winner: 0 12357
New Winner: 1 30144
New Winner: 0 115689
New Winner: 1 118857
New Winner: 0 15306
New Winner: 1 31722
New Winner: 0 116319
New Winner: 1 120060
New Winner: 0 14373


New Winner: 1 232122
New Winner: 0 133539
New Winner: 1 280662
New Winner: 0 152766
New Winner: 1 236457
New Winner: 0 212118
New Winner: 1 251145
New Winner: 0 301305
New Winner: 1 352704
New Winner: 0 119301
New Winner: 1 124788
New Winner: 0 189687
New Winner: 1 201729
New Winner: 0 340455
New Winner: 1 356580
New Winner: 0 104385
New Winner: 1 124686
New Winner: 0 193305
New Winner: 0 330744
New Winner: 1 355689
New Winner: 0 104313
New Winner: 1 147417
New Winner: 0 189582
New Winner: 1 196110
New Winner: 0 333183
New Winner: 1 354348
New Winner: 0 40671
New Winner: 0 182778
New Winner: 1 196101
New Winner: 0 245781
New Winner: 1 337326
New Winner: 0 26652
New Winner: 0 168771
New Winner: 1 182730
New Winner: 0 17010
New Winner: 0 154944
New Winner: 1 171000
New Winner: 0 12378
New Winner: 0 147294
New Winner: 1 165861
New Winner: 0 143349
New Winner: 1 161691
New Winner: 0 133494
New Winner: 1 149643
New Winner: 0 9708
New Winner: 0 116775
New Winner: 1 137271
New Winner: 0 88389

New Winner: 1 297525
New Winner: 0 88773
New Winner: 1 126321
New Winner: 0 153600
New Winner: 0 249123
New Winner: 1 286890
New Winner: 0 85458
New Winner: 1 114942
New Winner: 0 153369
New Winner: 0 259716
New Winner: 1 295584
New Winner: 0 85443
New Winner: 1 122841
New Winner: 0 153888
New Winner: 0 262527
New Winner: 1 305544
New Winner: 0 44943
New Winner: 1 155979
New Winner: 0 152913
New Winner: 0 253752
New Winner: 1 295713
New Winner: 0 36909
New Winner: 1 119856
New Winner: 0 152781
New Winner: 0 255363
New Winner: 1 298599
New Winner: 0 31278
New Winner: 1 87417
New Winner: 0 154515
New Winner: 0 292278
New Winner: 1 327360
New Winner: 0 29040
New Winner: 1 76308
New Winner: 0 155238
New Winner: 0 270486
New Winner: 1 309420
New Winner: 0 30906
New Winner: 1 32541
New Winner: 0 161754
New Winner: 1 163989
New Winner: 0 187851
New Winner: 1 263451
New Winner: 0 31020
New Winner: 0 151539
New Winner: 1 153147
New Winner: 0 16737
New Winner: 0 25485
New Winner: 1 35319
New Win

New Winner: 0 32217
New Winner: 1 79806
New Winner: 0 271230
New Winner: 1 318771
New Winner: 0 96651
New Winner: 1 233862
New Winner: 0 29688
New Winner: 1 75141
New Winner: 0 266577
New Winner: 1 308145
New Winner: 0 96453
New Winner: 1 224682
New Winner: 0 28047
New Winner: 1 71100
New Winner: 0 261981
New Winner: 1 298461
New Winner: 0 97314
New Winner: 1 206436
New Winner: 0 25821
New Winner: 1 72867
New Winner: 0 259440
New Winner: 1 295368
New Winner: 0 97725
New Winner: 1 194541
New Winner: 0 28797
New Winner: 1 75726
New Winner: 0 264999
New Winner: 1 301014
New Winner: 0 97608
New Winner: 1 202311
New Winner: 0 31416
New Winner: 1 77835
New Winner: 0 266277
New Winner: 1 299247
New Winner: 0 96939
New Winner: 1 211581
New Winner: 0 30678
New Winner: 1 77883
New Winner: 0 264138
New Winner: 1 298737
New Winner: 0 96579
New Winner: 1 210648
New Winner: 0 28350
New Winner: 1 76728
New Winner: 0 261984
New Winner: 1 299544
New Winner: 0 97263
New Winner: 1 204471
New Winner: 0 28

...

New Winner: 0 15345
New Winner: 1 17397
New Winner: 0 244512
New Winner: 0 62505
New Winner: 0 14877
New Winner: 1 16713
New Winner: 0 245418
New Winner: 0 64152
New Winner: 0 15294
New Winner: 1 18015
New Winner: 0 243627
New Winner: 0 58011
New Winner: 0 15279
New Winner: 1 17052
New Winner: 0 245838
New Winner: 0 61734
New Winner: 0 15180
New Winner: 1 17196
New Winner: 0 245625
New Winner: 0 64260
New Winner: 0 14016
New Winner: 1 16545
New Winner: 0 243168
New Winner: 0 61380
New Winner: 0 15738
New Winner: 1 17193
New Winner: 0 244764
New Winner: 0 63348
New Winner: 0 15201
New Winner: 1 16791
New Winner: 0 243930
New Winner: 0 63882
New Winner: 0 14715
New Winner: 1 16332
New Winner: 0 240135
New Winner: 0 62664
New Winner: 0 14586
New Winner: 1 16695
New Winner: 0 246375
New Winner: 0 61479
New Winner: 0 14889
New Winner: 1 16938
New Winner: 0 238152
New Winner: 0 64743
New Winner: 0 14805
New Winner: 1 16803
New Winner: 0 237936
New Winner: 0 62136
New Winner: 0 14256
New Winn

New Winner: 1 55650
New Winner: 0 234618
New Winner: 1 300309
New Winner: 0 90780
New Winner: 1 96696
New Winner: 0 19689
New Winner: 1 59091
New Winner: 0 238347
New Winner: 1 307233
New Winner: 0 90459
New Winner: 1 96561
New Winner: 0 19704
New Winner: 1 61722
New Winner: 0 236199
New Winner: 1 305424
New Winner: 0 90438
New Winner: 1 96975
New Winner: 0 20268
New Winner: 1 62676
New Winner: 0 235050
New Winner: 1 302748
New Winner: 0 91113
New Winner: 1 96744
New Winner: 0 20214
New Winner: 1 64809
New Winner: 0 238329
New Winner: 1 306351
New Winner: 0 90579
New Winner: 1 95991
New Winner: 0 19794
New Winner: 1 64545
New Winner: 0 236538
New Winner: 1 306309
New Winner: 0 90207
New Winner: 1 95406
New Winner: 0 19128
New Winner: 1 65010
New Winner: 0 236901
New Winner: 1 306540
New Winner: 0 90618
New Winner: 1 95379
New Winner: 0 19989
New Winner: 1 62640
New Winner: 0 237846
New Winner: 1 307665
New Winner: 0 90111
New Winner: 1 94884
New Winner: 0 20148
New Winner: 1 65208
New 

New Winner: 1 49542
New Winner: 0 244491
New Winner: 0 27081
New Winner: 1 50328
New Winner: 0 242307
New Winner: 0 27081
New Winner: 1 50328
New Winner: 0 242307
New Winner: 0 26700
New Winner: 1 52419
New Winner: 0 244887
New Winner: 0 25974
New Winner: 1 51870
New Winner: 0 240396
New Winner: 0 25194
New Winner: 1 49521
New Winner: 0 243567
New Winner: 0 26157
New Winner: 1 50115
New Winner: 0 240747
New Winner: 0 26469
New Winner: 1 47859
New Winner: 0 239574
New Winner: 0 25638
New Winner: 1 50607
New Winner: 0 241899
New Winner: 0 24579
New Winner: 1 49479
New Winner: 0 242715
New Winner: 0 25608
New Winner: 1 48780
New Winner: 0 241950
New Winner: 0 25233
New Winner: 1 51156
New Winner: 0 241461
New Winner: 0 25644
New Winner: 1 49869
New Winner: 0 242388
New Winner: 0 23274
New Winner: 1 52053
New Winner: 0 240840
New Winner: 0 23379
New Winner: 1 48300
New Winner: 0 237006
New Winner: 0 23379
New Winner: 1 48300
New Winner: 0 237006
New Winner: 0 24978
New Winner: 1 47796
New 

New Winner: 0 51573
New Winner: 1 57825
New Winner: 0 60543
New Winner: 1 68703
New Winner: 0 60300
New Winner: 0 58380
New Winner: 0 66087
New Winner: 1 75192
New Winner: 0 63540
New Winner: 0 62490
New Winner: 0 74628
New Winner: 1 98286
New Winner: 0 60981
New Winner: 0 56898
New Winner: 0 82800
New Winner: 1 106212
New Winner: 0 70809
New Winner: 0 57687
New Winner: 0 106998
New Winner: 1 155757
New Winner: 0 65292
New Winner: 0 50562
New Winner: 0 108684
New Winner: 1 159153
New Winner: 0 71217
New Winner: 0 54213
New Winner: 0 103926
New Winner: 1 153753
New Winner: 0 74658
New Winner: 0 56598
New Winner: 0 105771
New Winner: 1 151962
New Winner: 0 71814
New Winner: 0 62415
New Winner: 0 103689
New Winner: 1 150966
New Winner: 0 70713
New Winner: 0 64791
New Winner: 0 99114
New Winner: 1 148833
New Winner: 0 72144
New Winner: 0 62709
New Winner: 0 98031
New Winner: 1 144411
New Winner: 0 72885
New Winner: 0 64944
New Winner: 0 101298
New Winner: 1 149334
New Winner: 0 75714
New W

New Winner: 0 59442
New Winner: 1 61131
New Winner: 0 80097
New Winner: 1 92202
New Winner: 0 100314
New Winner: 1 137775
New Winner: 0 239316
New Winner: 1 269847
New Winner: 0 98439
New Winner: 1 127500
New Winner: 0 209826
New Winner: 1 228435
New Winner: 0 108087
New Winner: 1 142689
New Winner: 0 89850
New Winner: 1 104430
New Winner: 0 83922
New Winner: 1 99996
New Winner: 0 76509
New Winner: 1 101751
New Winner: 0 81624
New Winner: 1 107274
New Winner: 0 102711
New Winner: 1 132750
New Winner: 0 121062
New Winner: 1 153459
New Winner: 0 128112
New Winner: 1 192204
New Winner: 0 167172
New Winner: 1 202002
